In [ ]:
import torch
from torch import nn
from torch.autograd import grad
from torch import pi, cos, sin
import numpy as np
from copy import deepcopy

import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
torch.set_default_tensor_type(torch.FloatTensor)
torch.manual_seed(2023)

plt.rcParams['font.size'] = 15

In [ ]:
# Geometries
L = 10
r = 1

''' Determine the number of integration points by testing various dx and dy '''
dx = 0.1
dy = 0.1

''' Enter the thermal properties '''
q0 = 1.5
k = 0.384
alpha = 5*10**(-6)
T0 = 300

''' Enter the thermal properties '''
# Mechanical properties
E = 7*10**4
v = 0.25
P = 25
lam = v*E/(1-v**2)
mu = E/(2*(1+v))
gamma = 2*alpha*(lam+mu)

# domain points
Nx = int(L/dx)
Ny = int(L/dy)
xv = torch.linspace(0., L, Nx).to(device)
yv = torch.linspace(0., L, Ny).to(device)
xx, yy = torch.meshgrid([xv,yv], indexing='ij')
x = xx.reshape(-1)
y = yy.reshape(-1)
dom = torch.stack([x,y],dim=1)
doi = dom[:,0]**2 + dom[:,1]**2 >= r**2
dom = dom[doi,:]
dom.requires_grad = True


# boundary points
N_surf = int(Nx/2)
''' Determine x_hole and y_hole, the size of both is [N_surf X 1] '''
''' Don't forget to assign them to the device '''
x_hole = 
y_hole = 
hole = torch.stack([x_hole, y_hole], dim=1)
hole.requires_grad = True


N_surf = Ny
''' Determine x_right and y_right, the size of both is [N_surf X 1] '''
''' Don't forget to assign them to the device '''
y_right = 
x_right = 

right = torch.stack([x_right, y_right], dim=1)
right.requires_grad = True

# Schematics
fig,ax = plt.subplots(1,1, figsize=(10,5))
ax.scatter(dom[:,0].detach().cpu().numpy(), dom[:,1].detach().cpu().numpy(), s=1, c='k')
ax.scatter(right[:,0].detach().cpu().numpy(), right[:,1].detach().cpu().numpy(), s=1, c='b')
ax.scatter(hole[:,0].detach().cpu().numpy(), hole[:,1].detach().cpu().numpy(), s=1, c='r')
ax.set_aspect('equal')
print('# of total points:' + str(dom.shape[0]))

In [ ]:
# Temperature & temperature gradient
def GetT(x):
    T = model_T(x)
    ''' Enter an appropriate form of temperature field '''
    T *= ().unsqueeze(1)
    return T

def gradT(T,x):
    ''' Obtain the gradient of temperature '''
    dT = 
    return dT

# Define energy
def IntEnergy_th(T,x,volume):
    dT = gradT(T,x)
    ''' Enter an appropriate form of internal thermal energy '''
    psi_int = 
    U = psi_int.mean() * volume
    return U

def ExtWork_th(Ts,area):
    ''' Enter an appropriate form of external heat '''
    psi_heat = 
    W = psi_heat.mean() * area
    return W

In [ ]:
# Define model and optimizer
class NN(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_dim, num_layers):
        super().__init__()
        self.fcs = nn.ModuleList()
        self.acts = nn.ModuleList()

        for i in range(1, num_layers + 1):
            if i < num_layers:
                out_dim_ = hidden_dim
            else:
                out_dim_ = out_dim

            fc = nn.Linear(in_dim, out_dim_)

            self.fcs.append(fc)
            in_dim = hidden_dim

            if i < num_layers:
                self.acts.append(nn.Tanh())


    def forward(self, x):
        for fc, act in zip(self.fcs, self.acts):
            x = act(fc(x))
        return self.fcs[-1](x)

''' Determine the network configuration and optimizer '''
model_T = NN(in_dim= , out_dim= , hidden_dim= , num_layers= )
model_T = model_T.to(device)

optimizer = torch.optim.Adam(, lr=1e-3)

In [ ]:
Tc = 1

""" Write the volume of the domain and area of the surface """
volume = 
area = 

def train(max_iter):
    R_trace = []

    with tqdm(total=max_iter, desc='Training', unit='iter') as pbar:
        for t in range(1, max_iter+1):
            T = T0 + Tc * GetT(dom) # 무차원화를 위해 이렇게 변경함.
            
            ''' Enter an appropriate form of surface temperature '''
            Ts = Tc * GetT()
            
            U = IntEnergy_th(T, dom, volume)
            W = ExtWork_th(Ts, area)
            R = (U  - W)/T0
            
            R.backward(retain_graph=True)
            optimizer.step()
            optimizer.zero_grad()

            R_trace.append(R.item())

            if t % 10 == 0:
                pbar.set_postfix(loss=R)
                pbar.update(10)

    return R_trace

''' You may change the number of iterations '''
R_trace = train(4000)
    
fig,ax = plt.subplots(1,1, figsize=(5,5))
ax.plot(range(1, len(R_trace)+1), R_trace, 'b')
ax.set_yscale('symlog')

In [ ]:
# x and y coordinates
X = dom[:, 0]
Y = dom[:, 1]
X_np = X.detach().cpu().numpy()
Y_np = Y.detach().cpu().numpy()

''' Obtain the temperature gradients '''
T = T0 + Tc * GetT(dom)
dT = 

T = T.detach().cpu().numpy()
dT = dT.detach().cpu().numpy()

''' Write the components of dT '''
dTx = 
dTy = 

''' Plot the distribution of temperature and temperature gradient '''
fig = plt.figure(figsize=(24,6))
ax = fig.add_subplot(1, 3, 1)
ax.set_aspect('equal')
ax.set_title('T', fontdict = {'fontsize' : 30})
plt.scatter()
plt.colorbar()

ax = fig.add_subplot(1, 3, 2)
ax.set_aspect('equal')
ax.set_title('dTx', fontdict = {'fontsize' : 30})
plt.scatter()
plt.colorbar()

ax = fig.add_subplot(1, 3, 3)
ax.set_aspect('equal')
ax.set_title('dTy', fontdict = {'fontsize' : 30})
plt.scatter()
plt.colorbar()

In [ ]:
''' Obtain fixed temperature field '''
T = T0 + Tc * GetT(dom)
T_fixed = 

# Displacement, strain, stress
def GetU(x):
    u = model_u(x)
    ''' Enter an appropriate form of displacement field '''
    u[:,0] *= 
    u[:,1] *= 
    return u


def strain(u, x):
    du = grad(u[:,0].unsqueeze(1), x, torch.ones(x.shape[0],1).to(device), retain_graph=True, create_graph=True)[0]
    dv = grad(u[:,1].unsqueeze(1), x, torch.ones(x.shape[0],1).to(device), retain_graph=True, create_graph=True)[0]
    exx = du[:,0]
    eyy = dv[:,1]
    exy = du[:,1] + dv[:,0]
    e = torch.stack([exx, eyy, exy], dim=1)
    return e

''' Write the expressions of stress components '''
def stress(e,T):
    sxx =
    syy =
    sxy =
    s = torch.stack([sxx, syy, sxy], dim=1)
    return s

# Define energy
''' Write the expressions of internal energy '''
def IntEnergy_el(u,T,x,volume):
    e = strain(u,x)
    psi_int = 
    U = psi_int.mean() * volume
    return U

def ExtWork_el(us,area):
    psi_traction = us * P
    W = psi_traction.mean() * area
    return W

# Displacement network
''' Determine the network configuration and optimizer '''
model_u = NN(in_dim= , out_dim= , hidden_dim= , num_layers= )
model_u = model_u.to(device)

optimizer = torch.optim.Adam(, lr=1e-3)

In [ ]:
uc = 1e-4

""" Write the volume of the domain and area of the surface """
volume = 
area = 

def train(max_iter):
    R_trace = []

    with tqdm(total=max_iter, desc='Training', unit='iter') as pbar:
        for t in range(1, max_iter+1):
            u = uc * GetU(dom)
            us = uc * GetU(right)[:,0]
            U = IntEnergy_el(u,T_fixed,dom,volume)
            W = ExtWork_el(us,area)
            R = (U-W)/E
            
            R.backward(retain_graph=True)
            optimizer.step()
            optimizer.zero_grad()

            R_trace.append(R.item())

            if t % 10 == 0:
                pbar.set_postfix(loss=R)
                pbar.update(10)

    return R_trace

R_trace = train(5000)
    
fig,ax = plt.subplots(1,1, figsize=(5,5))
ax.plot(range(1, len(R_trace)+1), R_trace, 'b')
ax.set_yscale('symlog')

In [ ]:
# x and y coordinates
X = dom[:, 0]
Y = dom[:, 1]
X_np = X.detach().cpu().numpy()
Y_np = Y.detach().cpu().numpy()

''' Obtain the strain and stress '''
u = uc * GetU(dom) 
e =
s =

u = u.detach().cpu().numpy()
e = e.detach().cpu().numpy()
s = s.detach().cpu().numpy()

''' Write the strain and stress components '''
ux = u[:, 0]
uy = u[:, 1]
u_mag = (ux**2 + uy**2)**0.5

exx =
eyy = 
exy = 

sxx = 
syy = 
sxy = 


# Plot results

# ux
fig = plt.figure(figsize=(24, 16))
ax = fig.add_subplot(3, 3, 1)
ax.set_aspect('equal')
ax.set_title('ux', fontdict = {'fontsize' : 30})
plt.scatter(X_np, Y_np, c=ux, s=5, cmap='jet')
plt.colorbar()

# uy
ax = fig.add_subplot(3, 3, 2)
ax.set_aspect('equal')
ax.set_title('uy', fontdict = {'fontsize' : 30})
plt.scatter(X_np, Y_np, c=uy, s=5, cmap='jet')
plt.colorbar()

# u_mag
ax = fig.add_subplot(3, 3, 3)
ax.set_aspect('equal')
ax.set_title('|U|', fontdict = {'fontsize' : 30})
plt.scatter(X_np, Y_np, c=u_mag, s=5, cmap='jet')
plt.colorbar()

''' Plot the distribution of strain and stress components '''

# exx
ax = fig.add_subplot(3, 3, 4)
ax.set_aspect('equal')
ax.set_title('exx', fontdict = {'fontsize' : 30})
plt.scatter()
plt.colorbar()

# eyy
ax = fig.add_subplot(3, 3, 5)
ax.set_aspect('equal')
ax.set_title('eyy', fontdict = {'fontsize' : 30})
plt.scatter()
plt.colorbar()

# exy
ax = fig.add_subplot(3, 3, 6)
ax.set_aspect('equal')
ax.set_title('exy', fontdict = {'fontsize' : 30})
plt.scatter()
plt.colorbar()

# sxx
ax = fig.add_subplot(3, 3, 7)
ax.set_aspect('equal')
ax.set_title('sxx', fontdict = {'fontsize' : 30})
plt.scatter()
plt.colorbar()

# syy
ax = fig.add_subplot(3, 3, 8)
ax.set_aspect('equal')
ax.set_title('syy', fontdict = {'fontsize' : 30})
plt.scatter()
plt.colorbar()

# sxy
ax = fig.add_subplot(3, 3, 9)
ax.set_aspect('equal')
ax.set_title('sxy', fontdict = {'fontsize' : 30})
plt.scatter()
plt.colorbar()